# Trabajo Práctico 2: Reseñas de Películas - Ensamble

## Grupo 11 - "Los Outliers"

- Castillo, Carlos
- Destefanis, Juan Pablo
- Gómez, Celeste

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df = pd.read_csv('../datasets/train-complejo.csv', index_col=0)
df.head()

,texto_preprocesado,num_neg,num_adj_neg,num_exclm,lang,sentimiento
ID,,,,,,
0,critico_NOUN mencionar_VERB oz_DET episodio_NO...,7,4,0,es,positivo
1,pequén_ADJ pequén_ADJ produccion_PROPN tecnica...,2,0,2,es,positivo
2,pense_VERB maravilloso_ADJ pasar_VERB tiempo_N...,2,0,1,es,positivo
3,basicamente_ADV familia_NOUN nino_NOUN pequeno...,0,0,4,es,negativo
4,amor_NOUN tiempo_NOUN pelicula_NOUN visualment...,1,2,0,es,positivo


In [23]:
from sklearn.model_selection import train_test_split

df['lang+sentimiento'] = df['lang'].astype(str) + df['sentimiento'].astype(str)

X_train, X_test, y_train, y_test = train_test_split(df['texto_preprocesado'],
                                                    df[['lang+sentimiento', 'sentimiento']],
                                                    stratify=df['lang+sentimiento'],
                                                    test_size=0.2,
                                                    random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                      y_train['sentimiento'],
                                                      stratify=y_train['lang+sentimiento'],
                                                     test_size=0.2,
                                                     random_state=42)

In [24]:
X_train.head()

ID
11688    consejero_NOUN cabeza_NOUN forma_NOUN peak_NOU...
41644    simplemente_ADV pelicula_NOUN deber_AUX peor_A...
22573    acabar_VERB comprar_VERB dvd_NOUN deber_AUX te...
48102    NO_calificadoel_VERB NO_nivel_NOUN NO_escritur...
32220    coger_VERB siquiera_NOUN gracioso_ADJ infeccio...
Name: texto_preprocesado, dtype: object

In [27]:
y_test = y_test['sentimiento']

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=20, max_df=0.8)
encoder = LabelEncoder()

X_train_trans = vectorizer.fit_transform(X_train)
X_valid_trans = vectorizer.transform(X_valid)
X_test_trans = vectorizer.transform(X_test)

y_train_trans = encoder.fit_transform(y_train)
y_valid_trans = encoder.transform(y_valid)
y_test_trans = encoder.transform(y_test)

In [29]:
y_train_trans

array([1, 0, 1, ..., 1, 0, 1])

In [30]:
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [45]:
xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.3,
    reg_alpha=0.4,
    reg_lambda=0.3,
    gamma=6,
    max_depth=2,
)

In [ ]:
xgb.fit(X_train_trans, y_train_trans, eval_set=[(X_valid_trans, y_valid_trans)])

In [47]:
from sklearn.metrics import classification_report

y_train_pred = xgb.predict(X_train_trans)

print(classification_report(y_train_pred, y_train_trans))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89     15361
           1       0.92      0.88      0.90     16638

    accuracy                           0.90     31999
   macro avg       0.90      0.90      0.90     31999
weighted avg       0.90      0.90      0.90     31999



In [50]:
y_test_pred = xgb.predict(X_test_trans)

print(classification_report(y_test_pred, y_test_trans))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      4772
           1       0.87      0.83      0.85      5228

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [51]:
rf = RandomForestClassifier(
    n_estimators=1000,
    criterion="gini",
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=8,
    oob_score=True,
    n_jobs=-1,
)

In [53]:
nb = MultinomialNB()

In [55]:
from sklearn.ensemble import StackingClassifier

estimators = [
    ('random-forest', rf),
    ('xgboost', xgb),
    ('naive-bayes', nb),
]

clf = StackingClassifier(
    estimators=estimators,
    cv=5,
    stack_method='auto',
    n_jobs=-1,
)

In [56]:
clf.fit(X_train_trans, y_train_trans)

StackingClassifier(cv=5,
                   estimators=[('random-forest',
                                RandomForestClassifier(max_depth=15,
                                                       min_samples_leaf=8,
                                                       min_samples_split=10,
                                                       n_estimators=1000,
                                                       n_jobs=-1,
                                                       oob_score=True)),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              e...
                                              interaction_constraints=None,
                                              learning_rate=0.3, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=2,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=500, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('naive-bayes', MultinomialNB())],
                   n_jobs=-1)

In [57]:
y_train_pred = clf.predict(X_train_trans)

print(classification_report(y_train_pred, y_train_trans))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91     15755
           1       0.92      0.90      0.91     16244

    accuracy                           0.91     31999
   macro avg       0.91      0.91      0.91     31999
weighted avg       0.91      0.91      0.91     31999



In [58]:
y_valid_pred = clf.predict(X_valid_trans)

print(classification_report(y_valid_pred, y_valid_trans))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      3898
           1       0.88      0.86      0.87      4102

    accuracy                           0.87      8000
   macro avg       0.87      0.87      0.87      8000
weighted avg       0.87      0.87      0.87      8000



In [59]:
y_test_pred = clf.predict(X_test_trans)

print(classification_report(y_test_pred, y_test_trans))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      4910
           1       0.88      0.86      0.87      5090

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [61]:
X_full_trans = vectorizer.fit_transform(df['texto_preprocesado'])
y_full_trans = encoder.fit_transform(df['sentimiento'])

In [62]:
clf.fit(X_full_trans, y_full_trans)

StackingClassifier(cv=5,
                   estimators=[('random-forest',
                                RandomForestClassifier(max_depth=15,
                                                       min_samples_leaf=8,
                                                       min_samples_split=10,
                                                       n_estimators=1000,
                                                       n_jobs=-1,
                                                       oob_score=True)),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              e...
                                              interaction_constraints=None,
                                              learning_rate=0.3, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=2,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=500, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('naive-bayes', MultinomialNB())],
                   n_jobs=-1)

In [65]:
kaggle_df = pd.read_csv('../datasets/test-complejo.csv', index_col=0)
kaggle_df['sentimiento'] = encoder.inverse_transform(clf.predict(vectorizer.transform(kaggle_df['texto_preprocesado'])))

In [66]:
kaggle_df['sentimiento']

ID
60000    negativo
60001    negativo
60002    negativo
60003    negativo
60004    negativo
           ...   
68594    positivo
68595    negativo
68596    negativo
68597    negativo
68598    negativo
Name: sentimiento, Length: 8599, dtype: object

In [67]:
kaggle_df['sentimiento'].value_counts()

sentimiento
positivo    5064
negativo    3535
Name: count, dtype: int64

In [68]:
kaggle_df['sentimiento'].to_csv('../predicciones/ensamble-nb-rf-xgb.csv')